# Family Tree Visual Image Generation

### Imports

In [19]:
import pandas as pd
import PIL
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import numpy as np
import os.path
from os import path
import textwrap
import math

import config #separate .py file where I have private data stored
# import exampleconfig as config
person0id = 'Rachel' #ID of person to set as focus for family tree
personsettings = config.personsettings[person0id]
print(personsettings)
person0 = personsettings['id']
print(personsettings['size_by_degree'])

{'id': 1, 'num_gens_down': 1, 'num_gens_up': 4, 'degrees_sideways_to_include': [2, 2, 1, 1, 1, 0], 'ex_list': [], 'incl_list': [[226, 1, 1, False], [227, 1, 1, False], [228, 1, 1, True], [229, 0, 2, False], [230, 0, 2, False], [231, 0, 2, False], [232, 1, 1, True]], 'size_by_degree': None, 'r': 300, 'canvas_width': 40, 'canvas_height': 20, 'top_border': 1, 'bottom_border': 5, 'side_border': 0.5, 'spacing_couple': 0.1, 'spacing_siblings': 0.2, 'min_spacing_generation': 0.3, 'min_spacing_family': 0.4, 'pic_aspect_ratio': 1.5, 'text_vert_fraction': 0.5, 'yspacing': 0.07, 'atextwrap': [22, 20, 20, 22, 22, 20, 22, 18, 20, 20, 20, 20, 24, 22, 24, 20], 'aframewratio': 1.5}
None


### Read Excel Family Database

In [20]:
#Set datatypes, using float instead of int because nan is float type
datatypes = {'ID':float, 'Person':str, 'Partner':str, 'PartnerRelationship':str, 
             'Father':float,'Mother':float,'BirthYear':float,'DeathYear':float,'Birthday':str,
             'AncestorText':str}

#Read family tree excel file into pandas dataframe
df = pd.read_excel('familytree.xlsx', dtype=datatypes, engine='openpyxl')
# df = pd.read_excel('harrypotterfamilytree.xlsx', dtype=datatypes, engine='openpyxl')
# df = pd.read_excel('khan.xlsx', dtype=datatypes, engine='openpyxl')
people = df["ID"].tolist()
df = df.set_index(['ID'])


### Functions

In [21]:
def getbirthyear(person):
    # Used when setting up initial dictionary of people from file
    # Cleans up null data and returns birth year
    value = df.at[person,'BirthYear']
    if math.isnan(value):
        value = 0
    else:
        value = int(value)
    return value

def getdeathyear(person):
    # Used when setting up initial dictionary of people from file
    # Cleans up null data and returns death year
    value = df.at[person,'DeathYear']
    if math.isnan(value):
        value = 0
    else:
        value = int(value)
    return value

def getancestortext(person):
    # Used when setting up initial dictionary of people from file
    # Cleans up null data and returns text about ancestors that is used along top row of tree
    value = df.at[person,'AncestorText']
    if isinstance(value,float) and math.isnan(value):
        value = ''
    return value

def parents(person):
    # Used when setting up initial dictionary of people from file
    # Returns IDs of father f and mother m for given person
    # Cleans up null data
    f = df.at[person,'Father']
    m = df.at[person,'Mother']
    if math.isnan(f):
        f = None
    if math.isnan(m):
        m = None
    return [f,m]

def getpartnerlist(person):
    # Used when setting up initial dictionary of people from file
    # Finds all listed IDs of partners. Delimited by ',' in excel file
    # Returns list of partner IDs
    partner = str(df.at[person,'Partner'])

    if partner == 'nan':
        return []
    partnerlist = []
    if len(partner) > 0: #
        partnerlist = partner.split(',')
        partnerlist = [int(x) for x in partnerlist]
    return partnerlist

def getsiblinglist(person, people):
    # Used when setting up initial dictionary of people from file
    # Checks if person shares parents with anyone in list and adds them as siblings
    # Returns list of sibling IDs
    f0,m0 = parents(person)
    siblist = []
    for p in people:
        f,m = parents(p)
        if p!=person and ((f is not None and f0==f) or (m is not None and m0==m)):
            siblist.append(p)
    return siblist

def nameperson(person):
    # Used when setting up initial dictionary of people from file
    # Cleans up null data and returns name based on ID
    name = df.at[person,'Person']
    return name





def birth_death_text(person):
    # Returns concatenated birth and death year text or padded spaces if either is blank
    byear = getbirthyear(person)
    if byear == 0: 
        byear = '       '
    dyear = getdeathyear(person)
    if dyear == 0: 
        dyear = '       '
    return f'{byear}-{dyear}'

def ispartner(person,person2):
    #Returns True if person and person2 are partners
    if str(person2) in famdict[person].partnerlist or person2 in famdict[person].partnerlist:
        return True
    return False

def issibling(person,sibling):
    #Returns True if person and sibling are siblings
    if str(sibling) in famdict[person].siblinglist or sibling in famdict[person].siblinglist:
        return True
    return False

def incoreline(person,person0,num_gens_up):
    #checks if person is in core line of ancestry of person0
    #core ancestry means being a parent, grandparent, greatgrandparent, etc. of person0
    #returns true if person in core line
    #returns f or m for if person is a father or a mother in the core line
    inline = False
    f_or_m = None
    idlist = [person0]
    newlist = []
    #Part of core line if person is person0 and top generation
    if person == person0 and num_gens_up == 0:
        inline = True
        f_or_m = 'f'
    ##looks up 5 generations to see if person in core line
    for i in range(6):
        for p in idlist:
            f = famdict[p].father
            m = famdict[p].mother
            if f == person and f is not None:
                inline = True
                f_or_m = 'f'
                return [inline,f_or_m]
            if m == person and m is not None:
                inline = True
                f_or_m = 'm'
                return [inline,f_or_m]
            if f is not None:
                newlist.append(f)
            if m is not None:
                newlist.append(m)
            
        idlist = newlist
    
    return [inline,f_or_m]


def getpersontree(person):
    #returns list of lists of ancestors up from person
    #loops through 5 generations and appends parents list to main list
    c = 6
    ptree = [[] for i in range(c)]
    ptree[0]= [person]
    for i in range(c-1):
        for p in ptree[i]:
            f = famdict[p].father
            if f is not None:
                ptree[i+1].append(f)
            m = famdict[p].mother
            if m is not None:
                ptree[i+1].append(m)
    return ptree

def readperson_position(person, person0):
    #compares common ancestors between person and person0, or person's partner and person0
    #based on how many generations up shared ancestor is, evaluate "vertical" and "sideways"
    #generation position of person. Vertical e.g. child = -1, cousin/sibling = 0, parent/aunt = 1
    #Sideways e.g. self, parent, grandparent, child = 0, sibling, aunt, nephew = 1, cousin, 2nd cousin = 2
    #Returns part = True if person is partner of somone related to person0 and vert,side position 
    #of their partner instead of them.
    #If not related, returns 100 for vert and side so person would be outside bounds
    vert = 100
    side = 100
    part = False
    p0tree = getpersontree(person0)
    ptree = getpersontree(person)
    
    ###search through generations for matching ancestor at differnet generation levels
    for i in range(len(ptree)): ##loop through generations of person tree
        for p in ptree[i]:  #loop through people in each list for a generation
            for x in range(len(p0tree)):  #loop through person tree for original person
                if p in p0tree[x]:
                    vert = x-i
                    if vert >= 0:
                        side = i
                    else:
                        side = x
                    return[vert,side,part]
            
    ###search for matching ancestor of person's spouse
    partnerlist = famdict[person].partnerlist
    for partnername in partnerlist:
        ptree = getpersontree(float(partnername))
        ###re-do search through generations for matching ancestor at differnet generation levels with partner name
        for i in range(len(ptree)): ##loop through generations of person tree
            for p in ptree[i]:  #loop through people in each list for a generation
                for x in range(len(p0tree)):  #loop through person tree for original person
                    if p in p0tree[x]:
                        vert = x-i
                        part = True
                        if vert >= 0:
                            side = i
                        else:
                            side = x
                        return[vert,side,part]
    return [vert, side, part]

def fathermotherlist(fmlist,person0):
    #given ancestor list of type e.g. ['f','f','m'] which means father of father of 
    #mother of person0, Returns ID of top ancestor to look for
    fmlist.reverse()
    person = person0
    for fm in fmlist:
        f = famdict[person].father
        m = famdict[person].mother
        if fm == 'f':
            person = f
        else:
            person = m
    return person

def stepsibling_list_check(newplist,plist):
    #Used during person list sorting process
    #Looks at IDs of all people in sorted list newplist and checks if
    #any people in remaining list plist are stepsiblings of anyone in newplist
    #places stepsiblings sorted into newplist and removes them from plist
    i=0
    while i < len(newplist):
        p = newplist[i]
        j = 0
        while j < len(plist):
            sib = plist[j]
            if isstepsibling(p,sib):
                incore = famdict[p].incore
                fmtype = famdict[p].corefm
                #if person is part of main generation chain and father then put step-sib left of whole family chain
                if (incore == True and fmtype == 'f'):
                    pindex = newplist.index(p)
                    ins = False #inserted indicator
                    while ins == False:
                        #if person at start of list, insert left
                        if pindex == 0:
                            if pindex <= i:
                                i = i-1
                            j = j-1
                            newplist.insert(pindex,sib) 
                            plist.remove(sib)
                            ins = True

                        #if person to left is person p's family then decrease index
                        elif newplist[pindex-1] in famdict[p].familylist:
                            pindex = pindex-1
                        else:
                            if pindex <= i:
                                i = i-1
                                j = j-1
                                newplist.insert(pindex,sib)
                                plist.remove(sib)
                                ins = True            
                #otherwise put all the way right of whole family chain
                else:
                    pindex = newplist.index(p)
                    ins = False #inserted indicator
                    while ins == False:
                        #if person at end of list, insert right
                        if pindex+1 == len(newplist):
                            if pindex+1 <= i:
                                i = i-1
                            j = j-1
                            newplist.append(sib) 
                            plist.remove(sib)
                            ins = True

                        #if person to right is in person p's family then increase index
                        elif newplist[pindex+1] in famdict[p].familylist:
                            pindex = pindex+1
                        else:
                            if pindex+1 <= i:
                                i = i-1
                            j = j-1
                            newplist.insert(pindex+1,sib)
                            plist.remove(sib)
                            ins = True
            j+=1
        i+=1             
    
def sibling_list_check(newplist,plist):
    #Used during person list sorting process
    #Looks at IDs of all people in sorted list newplist and checks if
    #any people in remaining list plist are siblings of anyone in newplist
    #places siblings sorted into newplist and removes them from plist
    #sorted location will be based on age (oldest to youngest left to right)
    #unless someone is father or mother in core line
    #then siblings get placed to left of father or right of mother
    i=0
    while i < len(newplist):
        p = newplist[i]
        j = 0
        while j < len(plist):
            sib = plist[j]
            if issibling(p,sib):
                incore = famdict[p].incore
                fmtype = famdict[p].corefm
                #if person p is part of main generation chain and father then put sibling left, even if younger
                #or if sibling older than person p put left
                if (incore and fmtype == 'f') or (not incore and famdict[sib].birthyear<famdict[p].birthyear):
                    pindex = newplist.index(p)
                    ins = False #inserted indicator
                    while ins == False:
                        #if person at start of list, insert left
                        if pindex == 0:
                            if pindex <= i:
                                i = i-1
                            j = j-1
                            newplist.insert(pindex,sib) 
                            plist.remove(sib)
                            ins = True

                        #if person to left is person p's partner then decrease index
                        elif ispartner(newplist[pindex-1],p):
                            pindex = pindex-1
                        else:
                            #if id left of person not also sibling 
                            if issibling(newplist[pindex-1],sib)==False: 
                                #if id left of person is spouse of sibling decrease index
                                if (pindex-1)>0:
                                    ppartners = famdict[newplist[pindex-1]].partnerlist
                                    sibcheck = 0
                                    #loop through partners of person left. If any of them are sibling to sib and 
                                    #located left of current person left, then move index down
                                    for ppartner in ppartners:
                                        if issibling(ppartner,sib) and ppartner in newplist[0:pindex-2]:
                                            sibcheck = 1
                                    if sibcheck == 1:
                                        pindex = pindex-1
                                    #if id left is not spouse of sibling already in left list then insert
                                    else:
                                        if pindex <= i:
                                            i = i-1
                                        j = j-1
                                        newplist.insert(pindex,sib)
                                        plist.remove(sib)
                                        ins = True
                                #if id left is not spouse of sibling already in left list then insert
                                else:
                                    if pindex <= i:
                                        i = i-1
                                    j = j-1
                                    newplist.insert(pindex,sib)
                                    plist.remove(sib)
                                    ins = True
                            #if id left of person is sibling then iterate    
                            else: 
                                #if older than sibling to left then move back and check conditions again
                                if famdict[sib].birthyear<famdict[newplist[pindex-1]].birthyear:
                                    pindex = pindex - 1 
                                else:
                                    if pindex <= i:
                                        i = i-1
                                    j = j-1
                                    newplist.insert(pindex,sib)
                                    plist.remove(sib)
                                    ins = True            
                #if person is part of main generation chain and mother then put sib right, even if older
                #or if sibling younger than or same age as person put right
                if (incore == True and fmtype == 'm') or (incore == False and famdict[sib].birthyear>=famdict[p].birthyear):
                    pindex = newplist.index(p)
                    ins = False #inserted indicator
                    while ins == False:
                        #if person at end of list, insert right
                        if pindex+1 == len(newplist):
                            if pindex+1 <= i:
                                i = i-1
                            j = j-1
                            newplist.append(sib) 
                            plist.remove(sib)
                            ins = True

                        #if person to right is person p's partner then increase index
                        elif ispartner(newplist[pindex+1],p):
                            pindex = pindex+1
                        else:
                            #if id right of person not also sibling 
                            if issibling(newplist[pindex+1],sib)==False: 
                                #if id right of person is spouse of sibling increase index
                                if (pindex+1)<len(newplist):
                                    ppartners = famdict[newplist[pindex+1]].partnerlist
                                    sibcheck = 0
                                    #loop through partners of person right. If any of them are sibling to sib and 
                                    #located right of current person right, then move index up
                                    for ppartner in ppartners:
                                        if issibling(ppartner,sib) and ppartner in newplist[pindex+1:len(newplist)-1]:
                                            sibcheck = 1
                                    if sibcheck == 1:
                                        pindex = pindex+1
                                    #if id right is not spouse of sibling already in right list then insert
                                    else:
                                        if pindex+1 <= i:
                                            i = i-1
                                        j = j-1
                                        newplist.insert(pindex+1,sib)
                                        plist.remove(sib)
                                        ins = True
                                #if id right is not spouse of sibling already in right list then insert
                                else:
                                    if pindex+1 <= i:
                                        i = i-1
                                    j = j-1
                                    newplist.insert(pindex+1,sib)
                                    plist.remove(sib)
                                    ins = True
                            #if id right of person is sibling then iterate    
                            else: 
                                #if younger than sibling to right then move right and check conditions again
                                if famdict[sib].birthyear>famdict[newplist[pindex+1]].birthyear:
                                    pindex = pindex + 1 
                                else:
                                    if pindex+1 <= i:
                                        i = i-1
                                    j = j-1
                                    newplist.insert(pindex+1,sib)
                                    plist.remove(sib)
                                    ins = True
            j+=1
        i+=1             
    
def partner_list_check(newplist,plist):
    #Used during person list sorting process
    #Looks at IDs of all people in sorted list newplist and checks if
    #any people in remaining list plist are partners of anyone in newplist
    #places partners sorted into newplist and removes them from plist
    i=0
    while i < len(newplist):
        p = newplist[i]
        partnerlist = famdict[p].partnerlist
        if len(partnerlist)>0:
            #check if one of the partners is in core line and which partner it is
            corepartner = None
            for partner in (partnerlist):
                #if partner is in core line then indicate record person
                if partner in plist: 
                    if famdict[partner].incore:
                        corepartner = partner
                        fmc = famdict[partner].corefm

            #if there is a corepartner
            if corepartner is not None:
                #if corepartner is mother then place right and all other partners left
                if fmc == 'm':
                    #if at end of list then append, otherwise insert
                    #add core mother right
                    if newplist.index(p)+1 <= i:
                        i = i-1
                    if newplist.index(p) + 1 == len(newplist):
                        newplist.append(corepartner)
                    else:
                        newplist.insert(newplist.index(p)+1,corepartner)
                    #add all other partners left
                    for partner in (partnerlist):
                        if partner in plist and partner != corepartner:
                            if newplist.index(p) <= i:
                                i = i-1
                            newplist.insert(newplist.index(p),partner)
                            plist.remove(partner)
                #if corepartner is father then place left and all other partners right
                if fmc == 'f':
                    #add core father left
                    if newplist.index(p) <= i:
                        i = i-1
                    newplist.insert(newplist.index(p),corepartner)
                    #add all other partners right
                    for partner in (partnerlist):
                        if partner in plist and partner != corepartner:
                            if newplist.index(p)+1 == len(newplist):
                                if newplist.index(p) <= i:
                                    i = i-1
                                newplist.append(partner)
                            else:
                                if newplist.index(p) <= i:
                                    i = i-1
                                newplist.insert(newplist.index(p)+1,partner)
                            plist.remove(partner)
                plist.remove(corepartner)

            #if there is no corepartner place first partner right, then stack rest left
            else:
                for partner in (partnerlist):
                    if partner in plist:
                        #place first partner on right then additional partners on left
                        #if end of list append right
                        if newplist.index(p)+1 == len(newplist):
                            if newplist.index(p)+1 <= i:
                                i = i-1
                            newplist.append(partner)
                        #if person to right of p is p's partner already then insert left
                        elif ispartner(p,newplist[newplist.index(p)+1]):
                            if newplist.index(p) <= i:
                                i = i-1
                            newplist.insert(newplist.index(p),partner)
                        #otherwise insert to right of p
                        else:
                            if newplist.index(p)+1 <= i:
                                i = i-1
                            newplist.insert(newplist.index(p)+1,partner)
                        plist.remove(partner)
        i+=1

def sortpersonlist(plist,num_gens_up,person0):
    #Beginning of sorting process given list of people. This function sorts the top/oldest
    #generation of people in the list. It starts by looking for first person on left based on
    #sorting from core family line up from person0. Father of father of father... goes on left
    #then mother on right. So left most person will be father father father..., right will be 
    #mother of mother of mother... however many generations up and all the rest in the middle 
    #accordingly. Once the first person in this father/motherlist is found, then you go through 
    #and look for all family members  to add next to that person. 
    #People are removed from the person list "plist" and added to "newplist" as they get sorted
    #partners get added for anyone already in new list
    #partners of partners get added for anyone already in new list
    #siblings get added for anyone already in new list
    #partners and partners of partners get added again after siblings added
    #then next person in father/mother list identified and added and loop through again
    
    newplist = [];

    ##array for sorting order for main branches based on father line from person0 'f' or mother line 'm'
    ##set up now to only handle 4 generations above (16 great-great-grandfathers), easily modified
    fmarray = [['f','f','f','f','f'],['f','m','f','f','f'],['f','f','m','f','f'],['f','m','m','f','f'],
               ['f','f','f','m','f'],['f','m','f','m','f'],['f','f','m','m','f'],['f','m','m','m','f'],
               ['f','f','f','f','m'],['f','m','f','f','m'],['f','f','m','f','m'],['f','m','f','f','m'],
               ['f','f','f','m','m'],['f','m','f','m','m'],['f','f','m','m','m'],['f','m','m','m','m']]
    if num_gens_up>0:
        fmlistlen = 2**(num_gens_up-1)
    else:
        fmlistlen = 0
    fmlist = fmarray[0:fmlistlen]
    fmlist = [x[0:num_gens_up] for x in fmlist]
    fm = 0
    
    while len(plist)>0:
        #find branch based on father mother line
        if num_gens_up>0:
            print(fmlist[fm])
            fmperson = fathermotherlist(fmlist[fm], person0)
            for p in plist:
                if p == fmperson:
                    newplist.append(p)
                    plist.remove(p)
                    break
        #if person of interest is top generation, there is no father/mother line
        #just start with first person in list
        else:
            newplist.append(plist[0])
            plist.remove(plist[0])
        
        ##check for partners
        partner_list_check(newplist,plist)
        print('after adding partners',plist,newplist)
    
        ##check for partners of partners by running again
        partner_list_check(newplist,plist)
        print('after adding partners partners',plist,newplist)
        
        ##check for siblings
        sibling_list_check(newplist,plist)
        print('after adding siblings',plist,newplist)          
        
        ##recheck for partners and partners of partners again now that siblings are added
        partner_list_check(newplist,plist)
        print('after adding partners after siblings',plist,newplist)
        
        ##check for partners of partners by running again
        partner_list_check(newplist,plist)
        print('after adding partners of partners after siblings',plist,newplist)
        ##iterate father/mother branch to next
        fm += 1
        
    return newplist

def sortpersonlist_lower(upperlist,plist,num_gens_up,g,person0):
    #Sort all lower generations below the top/oldest based on parents
    #go left to right through sorted uppergeneration list and look for children of parents
    #then go through same process as sorting upper generation adding partners and siblings
    #People are removed from the person list "plist" and added to "newplist" as they get sorted
    #partners get added for anyone already in new list
    #partners of partners get added for anyone already in new list
    #siblings get added for anyone already in new list
    #partners and partners of partners get added again after siblings added
    #then next person in father/mother list identified and added and loop through again
    
    newplist = [];
    
    while len(plist)>0:
        #loop through upper list in pairs looking for children
        for x in range(len(upperlist)):
            p1 = upperlist[x]
            childlist = []
            for j in range(len(plist)):

                p = plist[j]
                #if parents of person in list then add to temporary list 
                #to see if one is in core line
                f = famdict[p].father
                m = famdict[p].mother

                if (f==p1) or (m==p1):
                    childlist.append(p)

            #if only one child of parents then append them to end of new list
            if len(childlist) == 1:
                newplist.append(childlist[0])
                plist.remove(childlist[0])
                break
            #if multiple children
            elif len(childlist) > 1:
                #look for if any children in core line from person0, if so add that person 1st
                corechild = None
                for ch in childlist:
                    if famdict[ch].incore:
                        corechild = ch
                if corechild is not None:
                    newplist.append(corechild)
                    plist.remove(corechild)
                    break

                #if no core children, just add first child in list
                else:
                    newplist.append(childlist[0])
                    plist.remove(childlist[0])
                    break
            else:
                pass
        print('after adding children',plist,newplist)

        ##check for partners
        partner_list_check(newplist,plist)
        print('after adding partners',plist,newplist)

        ##check for partners of partners by running again
        partner_list_check(newplist,plist)
        print('after adding partners again',plist,newplist)

        ##check for siblings
        sibling_list_check(newplist,plist)             
        print('after adding siblings',plist,newplist)            

        ##check for step-siblings
        stepsibling_list_check(newplist,plist)             
        print('after adding step siblings',plist,newplist)  

        ##recheck for partners and partners of partners again now that siblings are added
        partner_list_check(newplist,plist)
        print('after adding partners after adding siblings',plist,newplist)

        ##check for partners of partners by running again
        partner_list_check(newplist,plist)
        print('after checking partners again after siblings',plist,newplist)
    return newplist

def numberfamilysetsingen(originallist):
    #find number of total familes in a given generation list 
    #standard family would be all siblings and parters
    #corefamily would be all siblings and partners of either person in core couple 
    #(parent or other ancestor of person0)
    copylist = originallist[:]
    numfams = 0

    for p in originallist:
#         print('originallist',originallist,'remove plist',copylist)
        if p in copylist:
            j=0
            while j < len(copylist):
                p2 = copylist[j]
                if p2 in famdict[p].familylist:
                    if p2 in copylist:
                        copylist.remove(p2)
                        j=j-1
                j+=1
                if j<0:
                    j=0
            if p in copylist:
                copylist.remove(p)
            numfams+=1
    return numfams

def spacebetween(person,personright,spacing_couple,spacing_siblings,space_between_families):
    #returns spacing in inches to be used between person and person on their right
    #based on type of relationship
    
    #if partner
    if ispartner(person,personright):
        return spacing_couple
    #if partner of partner of person to right
    partnerlist = famdict[personright].partnerlist
    for partner in partnerlist:
        if ispartner(person,partner):
            return spacing_couple
    #if sibling
    if issibling(person,personright):
        return spacing_siblings
    #if family, such as spouse of sibling
    if person in famdict[personright].familylist:
        return spacing_siblings
    return space_between_families

def getpartnertype(person,personright):
    #Returns partnertype between person and person to right
    partnerlist = famdict[person].partnerlist
    partnertypelist = famdict[person].partnertypelist
    for i,partner in enumerate(partnerlist):
        if partner == personright:
            return partnertypelist[i]
    return 'other'

def linebetween(person,personright,i,j,sortedpersonlist,r):
    #returns line data (start and end position, line width, line color)
    #for drawing line between person and person on right in partnerships
    #line color varies based on if partners are married, divorced, or other
    #line shifted slightly if person has >2 partners so as to not overlap with another partner
    linewidth = 5
    linegap_multiplepartners = 0.1*r
    line = None
    if ispartner(person,personright):
        xstart = int(r*(famdict[person].picx+famdict[person].picw))
        ystart = int(r*(famdict[person].picy+famdict[person].pich/2))
        xend = int(r*(famdict[personright].picx))
        yend = ystart
        mdo = getpartnertype(person,personright)
        if mdo =='married':
            linecolor = (238,221,164)
        elif mdo == 'divorced':
            linecolor = (158,31,99)
        else:
            linecolor = (0,255,0)
        line = [[(xstart,ystart),(xend,yend)],linecolor,linewidth]
        return line
    
    #if partners not right next to eachother because of other partners move line up
    partnerlist = famdict[personright].partnerlist
    for partner in partnerlist:
        if ispartner(person,partner) and partner in sortedpersonlist[i]:
            while True:
                k = 2
                person2 = sortedpersonlist[i][j+k]
                if ispartner(person,person2):
                    
                    xstart = int(r*(famdict[person].picx+famdict[person].picw))
                    ystart = int(r*(famdict[person].picy+famdict[person].pich/2)-
                                 linegap_multiplepartners*(k-1))
                    xend = int(r*(famdict[person2].picx))
                    yend = ystart
                    mdo = getpartnertype(person,person2)
                    if mdo =='married':
                        linecolor = (255,255,255)
                    elif mdo == 'divorced':
                        linecolor = (255,0,0)
                    else:
                        linecolor = (90,170,70)
                    line = [[(xstart,ystart),(xend,yend)],linecolor,linewidth]
                    return line
                else:
                    k+=1
    return line


def linetoparents(person,personlist,i,j,space_between_gens,r):
    #Returns line data (x,y positions, linewidth, line color) for lines between children and parents
    #Line goes straight up from top middle of child
    #Line goes straight down from middle x and middle y between 2 parents or straight down from bottom middle of 1 parent
    #if both parents are not on chart
    #line goes between child and parent at vertical position 1/4 of way up between generations (to account for nameplate)
    
    #picture size
    vert = famdict[person].vert
    side = famdict[person].side
    g = vert+num_gens_down
    
    linewidth = 5
#     linecolor = (255,255,255)
    linecolor = (238,221,164)
    line = None
    
    f = famdict[person].father
    m = famdict[person].mother

    if f in personlist[i+1] and m in personlist[i+1]:
        ###horizontal line from center of person over to between parents
        xperson = int(r*(famdict[person].picx + famdict[person].picw/2))
        for k,p in enumerate(personlist[i+1]):
            if p==f:
                xdad = famdict[p].picx + famdict[p].picw/2
                pic_h_parents = famdict[p].pich
            if p==m:
                xmom = famdict[p].picx + famdict[p].picw/2
                pic_h_parents = famdict[p].pich
        xparents = int(r*(min(xdad,xmom) + abs((xdad-xmom)/2)))
        
        y1 = int(r*(famdict[person].picy))
        ###vertically go up 1/4 way to next gen, can adjust later for better autospacing with nameplates
        y2 = int(r*(famdict[person].picy - (space_between_gens/4))) 
        y3 = int(r*(famdict[f].picy+pic_h_parents/2))
        
        ##line 1 position up from center of person to midline
        ##line 2 horizontal from center of person to middle of parents
        ##line 3 up from midline to center of parents
        line = [[(xperson,y1),(xperson,y2),(xparents,y2),(xparents,y3)],linecolor,linewidth] 
        return line  
    ##if only 1 parent listed in generation above
    if f in personlist[i+1] or m in personlist[i+1]:
        xperson = int(r*(famdict[person].picx + famdict[person].picw/2))
        if f in personlist[i+1]:
            p=f
        if m in personlist[i+1]:
            p=m
        pic_h_parent = famdict[p].pich
        pic_w_parent = famdict[p].picw
        xparents = int(r*(famdict[p].picx + famdict[p].picw/2))
        
        y1 = int(r*(famdict[person].picy))
        ###vertically go up 1/4 way to next gen, can adjust later for better autospacing with nameplates
        y2 = int(r*(famdict[person].picy - (space_between_gens/4))) 
        y3 = int(r*(famdict[p].picy+pic_h_parent+pic_w_parent/2))
        
        ##line 1 position up from center of person to midline
        ##line 2 horizontal from center of person to middle of parents
        ##line 3 up from midline to center of parents
        line = [[(xperson,y1),(xperson,y2),(xparents,y2),(xparents,y3)],linecolor,linewidth] 
        return line  
        
    return line

def getfontsize(fontfile,text,w):
    #Used to get font size for name and birth year text for nameplates
    #Iterates increasing font size until width of font fits within given width w * ratio factor
    #textratio. Decreases font back down 1 size at end. This could be done much faster with
    #bigger jump searches up and back down in font size, but the computation time here is 
    #insignificant in this code. 
    textratio = 0.82
    
    
    fontsize = 1
    font = ImageFont.truetype(fontfile, fontsize)


    breakpoint = w*textratio
    jumpsize = 75
    while True:
        if font.getsize(text)[0] < breakpoint:
            fontsize += jumpsize
        else:
            jumpsize = jumpsize // 2
            fontsize -= jumpsize
        font = ImageFont.truetype(fontfile, fontsize)
        if jumpsize <= 1:
            break
            
#     while font.getsize(text)[0] < w*textratio:
#         fontsize += 1
#         font = ImageFont.truetype(fontfile, fontsize)
    
#     fontsize -= 1

    return fontsize





def isstepsibling(person,sibling):
    #Check for stepsibling relationship between person and sibling
    #If father or mother of person is a partner of father or mother of sibling return True
    f = famdict[person].father
    m = famdict[person].mother
    if f != famdict[sibling].father and m != famdict[sibling].mother:
        if f is not None:
            if famdict[sibling].father in famdict[f].partnerlist or famdict[sibling].mother in famdict[f].partnerlist:
                return True

        if m is not None:
            if famdict[sibling].father in famdict[m].partnerlist or famdict[sibling].mother in famdict[m].partnerlist:
                return True
    return False


def samefamily(person1, person2):
    #Checks if 2 people are in same family
    #Standard family set would be all siblings and spouses of siblings
    #Core family set would be all siblings and spouses of siblings of both core parents 
    #(This is so there's no additional spacing between people in the families of core parents)
    #This does not currently cover every possible case that exists but is fairly extensive. 
    partners1 = famdict[person1].partnerlist
    partners2 = famdict[person2].partnerlist
    
    siblings1 = famdict[person1].siblinglist
    siblings2 = famdict[person2].siblinglist
    
    #person not same person
    if person1 == person2:
        return False
    #if person is sibling (e.g. Mom + Mom's brother)
    if issibling(person1,person2):
        return True
    #if person is partner (e.g. Mom + Dad)
    if ispartner(person1,person2):
        return True
    
    #if persons parents parters with person2 parents (step-sibling or half-sibling) 
    if famdict[person2].father is not None:
        if (famdict[person1].mother in famdict[famdict[person2].father].partnerlist):
            return True
    if famdict[person2].mother is not None:
        if (famdict[person1].father in famdict[famdict[person2].mother].partnerlist):
            return True
    
#     #if half-sibling where you share a parent
#     if famdict[person1].father == famdict[person2].father or famdict[person1].mother == famdict[person2].mother:
#         return True

    #partner's stepsibling's partner's partner (e.g. Mom's stepbrother's wife to step-mom)
    for partner1 in partners1:
        for partner2 in partners2:
            for part in famdict[partner2].partnerlist:
                if isstepsibling(partner1,part):
                    return True
    for partner2 in partners2:
        for partner1 in partners1:
            for part in famdict[partner1].partnerlist:
                if isstepsibling(partner2,part):
                    return True
    
    #partners parent partner with partner of parent of person2 (e.g. Mom's stepbrother's wife to Mom's half-brother)
    for partner1 in partners1:
        if famdict[partner1].father is not None:
            parts = famdict[famdict[partner1].father].partnerlist
            for part in parts:
                if famdict[person2].father in famdict[part].partnerlist:
                    return True
        if famdict[partner1].mother is not None:
            parts = famdict[famdict[partner1].mother].partnerlist
            for part in parts:
                if famdict[person2].mother in famdict[part].partnerlist:
                    return True
    for partner2 in partners2:
        if famdict[partner2].father is not None:
            parts = famdict[famdict[partner2].father].partnerlist
            for part in parts:
                if famdict[person1].father in famdict[part].partnerlist:
                    return True
        if famdict[partner2].mother is not None:
            parts = famdict[famdict[partner2].mother].partnerlist
            for part in parts:
                if famdict[person1].mother in famdict[part].partnerlist:
                    return True
    
    #if persons parents parters with person2 parents' partners (e.g. Mom's stepbrother to mom's half brother)
    if famdict[person1].father is not None:
        partnerlist = famdict[famdict[person1].father].partnerlist
        for partner in partnerlist:
            if (famdict[person2].father in famdict[partner].partnerlist):
                return True
    if famdict[person1].mother is not None:
        partnerlist = famdict[famdict[person1].mother].partnerlist
        for partner in partnerlist:
            if (famdict[person2].mother in famdict[partner].partnerlist):
                return True
    if famdict[person2].father is not None:
        partnerlist = famdict[famdict[person2].father].partnerlist
        for partner in partnerlist:
            if (famdict[person1].father in famdict[partner].partnerlist):
                return True
    if famdict[person2].mother is not None:
        partnerlist = famdict[famdict[person2].mother].partnerlist
        for partner in partnerlist:
            if (famdict[person1].mother in famdict[partner].partnerlist):
                return True
            
    #if persons parents partners with person2 partners' parents (e.g. Mom's stepbrother's wife to Mom's half-brother)
    for partner1 in partners1:
        if famdict[partner1].father is not None:
            if (famdict[person2].mother in famdict[famdict[partner1].father].partnerlist):
                return True
        if famdict[partner1].mother is not None:
            if (famdict[person2].father in famdict[famdict[partner1].mother].partnerlist):
                return True
    for partner2 in partners2:
        if famdict[partner2].father is not None:
            if (famdict[person1].mother in famdict[famdict[partner2].father].partnerlist):
                return True
        if famdict[partner2].mother is not None:
            if (famdict[person1].father in famdict[famdict[partner2].mother].partnerlist):
                return True
    
    #if persons parents parters with person2s partners parents (e.g. Mom's stepbrother's wife to Mom)
    if famdict[person1].father is not None:
        partnerlist = famdict[person2].partnerlist
        for partner in partnerlist:
            if (famdict[partner].mother in famdict[famdict[person1].father].partnerlist):
                return True
    if famdict[person1].mother is not None:
        partnerlist = famdict[person2].partnerlist
        for partner in partnerlist:
            if (famdict[partner].father in famdict[famdict[person1].mother].partnerlist):
                return True
    if famdict[person2].father is not None:
        partnerlist = famdict[person1].partnerlist
        for partner in partnerlist:
            if (famdict[partner].mother in famdict[famdict[person2].father].partnerlist):
                return True
    if famdict[person2].mother is not None:
        partnerlist = famdict[person1].partnerlist
        for partner in partnerlist:
            if (famdict[partner].father in famdict[famdict[person2].mother].partnerlist):
                return True
    
    ###partner steb siblings with partner of person (e.g. Mom's stepbrother's wife to Dad)
    for partner1 in partners1:
        for partner2 in partners2:
            if isstepsibling(partner1,partner2):
                return True
        
    
    ###partner stepsiblings with partner of sibling of person (e.g. Mom's stepbrother's wife to Dad's brother )
    for partner1 in partners1:
        for sib2 in siblings2:
            partnerlist = famdict[sib2].partnerlist
            for part in partnerlist:
                if isstepsibling(partner1,part):
                    return True
    for partner2 in partners2:
        for sib1 in siblings1:
            partnerlist = famdict[sib1].partnerlist
            for part in partnerlist:
                if isstepsibling(partner2,part):
                    return True
                
    ###partner stepsiblings with partner of sibling of partner of person (e.g. Mom's stepbrother's wife to Dad's brother's wife)
    for partner1 in partners1:
        for partner2 in partners2:
            for sib in famdict[partner2].siblinglist:
                for part in famdict[sib].partnerlist:
                    if isstepsibling(partner1,part):
                        return True
    for partner2 in partners2:
        for partner1 in partners1:
            for sib in famdict[partner1].siblinglist:
                for part in famdict[sib].partnerlist:
                    if isstepsibling(partner2,part):
                        return True
                    
    ##e.g. heidi to several people 
    
    ##Person1's sibling's partner is stepsibling with person2 (e.g. Mom's stepbrother to Dad's brother)
    for sibling1 in siblings1:
        for part in famdict[sibling1].partnerlist:
            if isstepsibling(part,person2):
                return True
    for sibling2 in siblings2:
        for part in famdict[sibling2].partnerlist:
            if isstepsibling(part,person1):
                return True
            
    ##Person's partner's, partner's sibling (e.g. Mom's brother to stepmom)
    for partner1 in partners1:
        for part in famdict[partner1].partnerlist:
            if issibling(part,person2):
                return True
    for partner2 in partners2:
        for part in famdict[partner2].partnerlist:
            if issibling(part,person1):
                return True
    
    ######sibling start
    #if person is partner of your sibling (e.g. Mom's brother to Dad)
    for sibling1 in siblings1:
        if sibling1 in partners2:
            return True
    for sibling2 in siblings2:
        if sibling2 in partners1:
            return True
    
    #if person is sibling of partner of your sibling (e.g. Mom's Brother + Dad's Sister)
    for sibling1 in siblings1:
        parts = famdict[sibling1].partnerlist
        for part in parts:
            if issibling(person2,part):
                return True
    for sibling2 in siblings2:
        parts = famdict[sibling2].partnerlist
        for part in parts:
            if issibling(person1,part):
                return True
    
    #if person is partner of sibling of partner of your sibling (e.g. Mom's brother + Dad's Sister's Husband)
    for sibling1 in siblings1:
        parts = famdict[sibling1].partnerlist
        for part in parts:
            sibs = famdict[part].siblinglist
            for sib in sibs:
                if ispartner(person2,sib):
                    return True
    for sibling2 in siblings2:
        parts = famdict[sibling2].partnerlist
        for part in parts:
            sibs = famdict[part].siblinglist
            for sib in sibs:
                if ispartner(person1,sib):
                    return True
                
    ######partner start
    #if person is partner of your partner (e.g. Grandpa + Step-grandpa)
    for partner1 in partners1:
        if ispartner(person2,partner1):
            return True
    for partner2 in partners2:
        if ispartner(person1,partner2):
            return True
        
    #if person is partner of your partner of your partner (e.g. Stepmom + Stepdad)
    for partner1 in partners1:
        parts = famdict[partner1].partnerlist
        for part in parts:
            if ispartner(part,person2):
                return True
    for partner2 in partners2:
        parts = famdict[partner2].partnerlist
        for part in parts:
            if ispartner(part,person1):
                return True
        
    #if person is sibling of your partner (e.g. Mom's brother's wife + Mom)
    for partner1 in partners1:
        if partner1 in siblings2:
            return True
    for partner2 in partners2:
        if partner2 in siblings1:
            return True
    
    #if person is partner of sibling of your partner (e.g. Mom's brother's wife + Dad)
    for partner1 in partners1:
        sibs = famdict[partner1].siblinglist
        for sib in sibs:
            if ispartner(person2,sib):
                return True
    for partner2 in partners2:
        sibs = famdict[partner2].siblinglist
        for sib in sibs:
            if ispartner(person1,sib):
                return True     
    
    #if person is sibling of partner of sibling of your partner (e.g. Mom's brother's wife + Dad's sister)
    for partner1 in partners1:
        sibs = famdict[partner1].siblinglist
        for sib in sibs:
            sibpartners = famdict[sib].partnerlist
            for sibpartner in sibpartners:
                if issibling(person2,sibpartner):
                    return True
    for partner2 in partners2:
        sibs = famdict[partner2].siblinglist
        for sib in sibs:
            sibpartners = famdict[sib].partnerlist
            for sibpartner in sibpartners:
                if issibling(person1,sibpartner):
                    return True
     
    #if person is partner of sibling of partner of sibling of your partner (e.g. Mom's brother's wife + Dad's Sister's Husband)
    for partner1 in partners1:
        sibs = famdict[partner1].siblinglist
        for sib in sibs:
            sibpartners = famdict[sib].partnerlist
            for sibpartner in sibpartners:
                sibs2 = famdict[sibpartner].siblinglist
                for sib2 in sibs2:
                    if ispartner(person2,sibs2):
                        return True
    for partner2 in partners2:
        sibs = famdict[partner2].siblinglist
        for sib in sibs:
            sibpartners = famdict[sib].partnerlist
            for sibpartner in sibpartners:
                sibs2 = famdict[sibpartner].siblinglist
                for sib2 in sibs2:
                    if ispartner(person1,sibs2):
                        return True
    return False

def getancfontsize(text,fontfile,w,h):
    #Same as getfontsize but for ancestor box text (using different textratio)
    wtextratio = 0.7
    htextratio = 0.7
    maxfontsize = 0
    
    #loop through several character wrapping values to get max font size
    for i in range(1,100):
        wrapper = textwrap.TextWrapper(width=i) 
        word_list = wrapper.wrap(text=text) 
        fontsize = 1
        
        fontunder = True
        
        while fontunder:
            font = ImageFont.truetype(fontfile, fontsize)
            fontheight = 0
            for wl in word_list: 
                if font.getsize(wl)[0] > w*wtextratio:
                    fontunder = False
                    break
                fontheight += font.getsize(wl)[1]
            if fontheight > h*htextratio:
                fontunder = False
            if fontunder and fontsize > maxfontsize:
                maxfontsize = fontsize
                wrap = i
            fontsize += 1
                             
    return wrap,maxfontsize

def partnertype(person1,person2):
    # Returns type of partnership between two people
    partner = df.at[person1,'Partner']
    rel = df.at[person1,'PartnerRelationship']
    partnerlist = []
    rellist = []

    try:
        partnerlist = [float(partner)]
        rellist = [rel]
    except :
#         partner = str(partner.item())
        partnerlist = partner.split(',')
        partnerlist = [int(x) for x in partnerlist]
        
        if isinstance(rel,pd.Series):
            if len(rel)==0:
                return 'other'
        
#         rel = rel.item()
        
        if isinstance(rel,str):
            rellist = rel.split(',')
        else:
            return 'other'
    
    for i,p in enumerate(partnerlist):
        if p==person2:
            if rellist[i] == 'M':
                return 'married'
            elif rellist[i] == 'D':
                return 'divorced'
            else:
                return 'other'
    return 'other'

### Set Up Person Dictionary

In [22]:
###############Set up person class to store data and loop through people to get information
class Person:
    def __init__(self,name,father,mother,partnerlist,partnertypelist,
                 siblinglist,incore,corefm,familylist,birthyear,deathyear,
                 birthdeathtext,ancestortext,vert,side,part,picw,pich,picx,picy):
        self.name = name #string of full name
        self.father = father #father ID
        self.mother = mother #mother ID
        self.partnerlist = partnerlist #list of partner IDs
        self.partnertypelist = partnertypelist #list of partner types (e.g. married, divorced)
        self.siblinglist = siblinglist #list of sibling IDs
        self.incore = incore #bool in core family line
        self.corefm = corefm #f or m for father or mother in core family line
        self.familylist = familylist #list of same generation family IDs
        self.birthyear = birthyear #birth year
        self.deathyear = deathyear #death year
        self.birthdeathtext = birthdeathtext #str birth and death year text for display
        self.ancestortext = ancestortext #ancestor text for display for top level generation
        self.vert = vert #vertical generation position vs. person0
        self.side = side #sideways familiar poistion vs. person0
        self.part = part #bool partner of family relation of person0
        self.picw = picw #width of picture for display
        self.pich = pich #height of picture for display
        self.picx = picx #x offset position for top left corner of picture for display from canvas top left corner
        self.picy = picy #y offset position for top left corner of picture for display from canvas top left corner
        
    def xOffset(self,x):
        self.picx = self.picx + x
    

#create dictionary to link person ID to person object instance 
#loop through all people and create person object with all data that can be loaded before knowing who person0 is
famdict = {}
for person in people:
    name = nameperson(person)
    father, mother = parents(person)
    partnerlist = getpartnerlist(person)
    
    partnertypelist = []
    for partner in partnerlist:
        partnertypelist.append(partnertype(person,partner))
    siblinglist = getsiblinglist(person,people)
    
    familylist = []
    incore = False
    corefm = None
    birthyear = getbirthyear(person)
    deathyear = getdeathyear(person)
    
    byear = birthyear
    if birthyear == 0:
            byear = '       '
    dyear = deathyear
    if deathyear == 0:
        dyear = '       '
    birthdeathtext = str(byear) + '-' + str(dyear)

    ancestortext = getancestortext(person)
    vert = 100
    side = 100
    part = False
    
    picw = 0
    pich = 0
    picx = 0
    picy = 0
    
    p=Person(name,father,mother,partnerlist,partnertypelist,siblinglist,
             incore,corefm,familylist,birthyear,deathyear,birthdeathtext,
             ancestortext, vert, side, part, picw, pich, picx, picy)
    famdict[person] = p

#loop through all people again now that sibling and partner relationships added 
#to create family list
for person in people:
    familylist = []
    for person2 in people:
        if samefamily(person,person2):
            familylist.append(person2)
    famdict[person].familylist = familylist




## Set Key Person and Find People to Include

In [23]:
#####################################################################
# Set up generations to show and key person
#####################################################################


#Set up details about which generations to include
#num_gens_down is number generations down to include from person0
#num_gens_up is number of generations up from person0
#num_gens is total generations including person0
#degrees_sideways_to_include is degrees sideways at each generation to include 
#starting from lowest generation
#Example with 3 gens up 1 gen down with degrees_sideways_to_include [1,2,3,1,0] would be
#great grandparents, grandparents + great aunts/uncles, parents + aunts/uncles + 
#parent cousins, siblings + cousins, children + nieces/nephews 

#Not just any combination of generations will work. You must have full set of parental core line
#up to top generation because sorting works from top generation down. 

#Some combinations will not visualize properly, e.g. cannot include 2nd cousins because sorting
#may split up core family and the way children need to be displayed below parents 
#to left and right of main couple

#ex_list id's listed in ex_list will not get included, regardless of relationship
#cannot exclude someone in core parentage line
#incl_list will include people who would't get included otherwise. You manually put in list of
#[id, vert, side, part] for each person in inclusion list. If stepsiblings, they'll get sorted
#properly in family list. If some other relation, might not get organized or show up right now 
#because of unknown family connection


num_gens_down = personsettings['num_gens_down'] #how many generations below key person to include
num_gens_up = personsettings['num_gens_up'] #how many generations above key person to include
num_gens = 1 + num_gens_down + num_gens_up #how many generations total including gen of key person
degrees_sideways_to_include = personsettings['degrees_sideways_to_include']
ex_list = personsettings['ex_list']
incl_list = personsettings['incl_list'] #inclusion list for step kids [id,vert,side,part]


#Photo scaling by generation
######Photo Scaling
##if you want variable photo sizing by generation or degree sideway
#list by generation starting with bottom generation
#then in each generation list by degree sideways
#e.g. size_by_degree=[[1.2,1,0.8],[1.2,1],[1,0.8],[1.2],[1]]
#must have a size listed for all generations/sideways in your generation setup
if personsettings['size_by_degree'] is not None:
    size_by_degree = personsettings['size_by_degree']
else:
    ##if you want equal sizing of all photos
    size_by_degree = np.full((num_gens,max(degrees_sideways_to_include)+1),1)


#####################################################################
# Find tree of people to include
#####################################################################

#set up empty lists
num_couples = [0]*num_gens
num_siblings = [0]*num_gens
num_people = [0]*num_gens
photo_scaling = [0]*num_gens
max_vertical_scaling = [0]*num_gens
num_peopleinstep = np.zeros((num_gens,max(degrees_sideways_to_include)+1))
space_between_families = [0]*num_gens
num_families = [0]*num_gens
personlist = [[] for i in range(num_gens)]
namelist = [[] for i in range(num_gens)]
sortedpersonlist = [[] for i in range(num_gens)]

#Loop through all people and record whether person is in core line
for p in people:
    incore,f_or_m = incoreline(p,person0,num_gens_up)
    famdict[p].incore = incore
    famdict[p].corefm = f_or_m

#loop through all people and assign vert, side 
#find ones that fit vertical and sideways criteria to include on tree
#keep track of amount of people in various parts of generation 
#and max size scaling in each generation 
for i,p in enumerate(people):
    vert, side, part = readperson_position(p, person0)
    
    #look for people in inclusion list and manually put in vert,side data
    for inc in incl_list:
        if p==inc[0]:
            vert = inc[1]
            side = inc[2]
            part = inc[3]
    #update person class instance
    famdict[p].vert = vert
    famdict[p].side = side
    famdict[p].part = part
    
    if p not in ex_list: #don't include ids in exclusion list
        g = vert + num_gens_down #generation position in lists
        #if person fits in selected generation range
        if vert <= num_gens_up and vert >= -1*num_gens_down:
            if side <= degrees_sideways_to_include[g]:
                #if the person is only a partner of someone in the generation, 
                
                #only include them if the children generation included
                if part == True:
                    #for lower generations, sideways same degree as parents
                    if vert <= 0:
                        if side <= degrees_sideways_to_include[g-1]:
                            personlist[g].append(p)
                            namelist[g].append(famdict[p].name)
                            num_people[g] += 1
                            num_peopleinstep[g][side] += 1
                            if size_by_degree[g][side] > max_vertical_scaling[g]:
                                max_vertical_scaling[g] = size_by_degree[g][side]
                    #for upper generations, sideways one degree off of parents
                    else:
                        if side <= max(0,degrees_sideways_to_include[g-1]-1):
                            personlist[g].append(p)
                            namelist[g].append(famdict[p].name)
                            num_people[g] += 1
                            num_peopleinstep[g][side] += 1
                            if size_by_degree[g][side] > max_vertical_scaling[g]:
                                max_vertical_scaling[g] = size_by_degree[g][side]
                else:
                    personlist[g].append(p)
                    namelist[g].append(famdict[p].name)
                    num_people[g] += 1
                    num_peopleinstep[g][side] += 1
                    if size_by_degree[g][side] > max_vertical_scaling[g]:
                                max_vertical_scaling[g] = size_by_degree[g][side]
print('personIDlist',personlist) 
# print('namelist',namelist)
print('num_people',num_people)

#find number of couples
for i in range(num_gens):
    for person in personlist[i]:
        partnerlist = famdict[person].partnerlist
        for partnername in partnerlist:
            if float(partnername) in personlist[i]:
                num_couples[i] += 0.5
print('num_couples',num_couples)


#find number of siblings
sibsfound = []
for i in range(num_gens):
    for person in personlist[i]:
        for person2 in personlist[i]:
            if person not in sibsfound and person2 not in sibsfound and person2 in famdict[person].siblinglist:
                num_siblings[i] += 1
                sibsfound.append(person2)
print('num_siblings',num_siblings)

            
########sort names
#sort upper generation
sortedpersonlist[num_gens-1] = sortpersonlist(personlist[num_gens-1],num_gens_up,person0)

#sort lower generations
for i in range(num_gens-1):
    upperlist = sortedpersonlist[num_gens-1-i]
    lowerlist = personlist[num_gens-2-i]
    sortedlowerlist = sortpersonlist_lower(upperlist,lowerlist, num_gens_up,i,person0)
    sortedpersonlist[num_gens-2-i] = sortedlowerlist

print('sorted list',sortedpersonlist)


#number of families (sets of siblings/spouses) in a generation
for i in range(len(sortedpersonlist)):
    num_families[i] = numberfamilysetsingen(sortedpersonlist[i])
print('numfamilies',num_families)


###################################################################################
# IF SORTING DOES NOT WORK PROPERLY AND YOU CAN'T GET IT TO WORK FOR YOUR SITUATION
# you could hand provide a sorted list of lists of each generation and some of the needed numbers for below like 
# number of families, partners, etc. That might work, but maybe not, depending on why the sorting is failing.

personIDlist [[0.0, 5.0, 201.0, 203.0, 206.0, 207.0, 209.0, 210.0, 211.0, 213.0, 214.0, 215.0, 216.0], [1.0, 2.0, 3.0, 4.0, 21.0, 22.0, 26.0, 33.0, 34.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 202.0, 204.0, 205.0, 208.0, 212.0, 229.0, 230.0, 231.0], [6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 27.0, 28.0, 29.0, 30.0, 31.0, 193.0, 197.0, 226.0, 227.0, 228.0, 232.0], [17.0, 18.0, 19.0, 20.0, 23.0, 35.0, 187.0, 188.0, 189.0, 190.0, 192.0, 194.0, 195.0, 196.0, 198.0, 199.0, 200.0], [54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 191.0], [233.0, 234.0, 235.0, 236.0, 237.0, 238.0, 239.0, 240.0, 241.0, 242.0, 243.0, 244.0, 245.0, 246.0, 247.0, 248.0]]
num_people [13, 31, 20, 17, 9, 16]
num_couples [0, 12.0, 9.0, 6.0, 5.0, 8.0]
num_siblings [5, 12, 9, 9, 0, 0]
['f', 'f', 'f', 'f']
after adding partners [235.0, 236.0, 237.0, 238.0, 239.0, 240.0, 241.0, 242.0, 243.0, 244.0, 245.0, 246.0, 247.0, 248.0] [233.0, 234]
after adding partners partners

In [24]:
#####################################################################
# Set up canvas details
#####################################################################
r = personsettings['r'] #resolution dpi

#provide filepath where main files stored or link to separate configuration file
# filedir = 'C:\\Users\\User\\Documents\\Python Scripts\\Family Tree'
filedir = config.filedirectory

#Set all canvas details,spacing, and other formatting details
#all in inches
## canvas_width - total canvas width
## canvas_height - total canvas height
## top_border - canvas top border
## bottom_border - canvas bottom border
## side_border - canvas side borders (border for single side)
## spacing_couple - spacing between partners
## spacing_siblings - spacing between siblings
## min_spacing_generation - minimum vertical spacing between generations
#  if vertical picture size not limiting dimension, this spacing will be 
#  calculated automatically 
## min_spacing_family - minimum spacing between family sets in same generation
#  if horizontal picture size not limiting dimenion, this spacing will be
#  calculated automatically. This will be used for generation with most people
#  Other generations will get evenly spaced horizontally
## pic_aspect_ratio - aspect ratio of pictures to be used. Square pics would be 1
## text_vert_fraction - fraction of picture width to use for nameplate height
## yspacing - amount to shift parent-child lines up if they are overlapping
## atextwrap - ancestor text at top of frame will get wrapped according to the 
#  character lenths in this list. Manually edit based on the length of ancestor 
#  text. 
## aframeratio - ancestor text frame multiplier by image width

#canvas details all in inches
canvas_width = personsettings['canvas_width']
canvas_height = personsettings['canvas_height']
top_border = personsettings['top_border']
bottom_border = personsettings['bottom_border'] 
side_border = personsettings['side_border']
spacing_couple = personsettings['spacing_couple']
spacing_siblings = personsettings['spacing_siblings']
min_spacing_generation = personsettings['min_spacing_generation']
min_spacing_family = personsettings['min_spacing_family']
pic_aspect_ratio = personsettings['pic_aspect_ratio']
text_vert_fraction = personsettings['text_vert_fraction']
yspacing = personsettings['yspacing']*r #amount to shift lines up if they are overlapping
atextwrap = personsettings['atextwrap'] #character width for ancestor text wrap
aframewratio = personsettings['aframewratio'] #ancestor text frame multiplier by image width


#ttf file of font to use
fontfile = filedir + '\\Fonts\\Redressed\\Redressed-Regular.ttf'
# fontfile = filedir + '\\Fonts\\italianno\\Italianno-Regular-OTF.otf'
# fontfile = filedir + '\\Fonts\\Chantelli-Antiqua\\Chantelli_Antiqua.ttf'
# fontfile = filedir + '\\Fonts\\Advent_Pro\\AdventPro-SemiBold.ttf'


#####################################################################
# Picture Location Spacing and Sizing
#####################################################################

#####find limits in order to determine base sizing of photos
#for even or variable sizing of photos, find maximum size photo can be based on sizing scaling
#and vertical constraint from number of generations
#or horizontal constraint based on number of people in generation and all required spacings
#vertical limits
vertical_spaces = top_border + bottom_border + (num_gens - 1)*min_spacing_generation

max_vertical = (canvas_height - vertical_spaces)/(sum(max_vertical_scaling)*pic_aspect_ratio*
                                                  (1+text_vert_fraction))
pic_size = max_vertical

#horizontal limits
for i in range(len(num_peopleinstep)):
    tot = 0
    for j in range(degrees_sideways_to_include[i]+1):
        tot += num_peopleinstep[i][j]*size_by_degree[i][j]
    photo_scaling[i] = tot
print('photo_scaling',photo_scaling)

for i in range(0,num_gens):
    if num_people[i]>0:
        horizontal_spaces = (side_border*2 + (num_couples[i]*spacing_couple) + (num_siblings[i]*spacing_siblings) + 
                            (num_families[i]*min_spacing_family))
        max_horizontal = (canvas_width-horizontal_spaces)/photo_scaling[i]
        if max_horizontal < pic_size:
            pic_size = max_horizontal

pic_size_w = pic_size
pic_size_h = pic_size*pic_aspect_ratio
print('pic_width',pic_size, 'pic_height',pic_size_h)


#spacing between family sets of siblings is remainder of space divided evenly 
for i in range(num_gens):
    horizontal_spaces = (side_border*2 + (num_couples[i]*spacing_couple) + (num_siblings[i]*spacing_siblings))
    pic_spaces = 0
    for j in range(degrees_sideways_to_include[i]+1):
        pic_spaces += num_peopleinstep[i][j]*size_by_degree[i][j]*pic_size_w
    space_between_families[i] = (canvas_width - horizontal_spaces - pic_spaces)/(num_families[i])
print('space_between_families',space_between_families)

#spacing between generations
vertical_spaces = top_border + bottom_border
pic_spaces = 0
for i in range(num_gens):
    maxpic = 0
    for j in range(degrees_sideways_to_include[i]+1):
        if num_peopleinstep[i][j]>0:
            vsize = size_by_degree[i][j]*pic_size_h
            if maxpic<vsize:
                maxpic = vsize
    pic_spaces += maxpic
space_between_gens = (canvas_height - vertical_spaces - pic_spaces)/(num_gens-1)
print('space_between_gens',space_between_gens)

############x,y offsets for picture positioning
pic_x_offset = [[] for i in range(num_gens)]
pic_y_offset = [[] for i in range(num_gens)]


######x offsets
#go along generation left to right and assign x position to person
for i in range(num_gens):
    pic_x_offset[i] = np.zeros(len(sortedpersonlist[i]))
    pic_y_offset[i] = np.zeros(len(sortedpersonlist[i]))
    
    #on left start with half space between families for first person
    x = side_border + space_between_families[i]/2
    pic_x_offset[i][0] = x
    for j in range(len(sortedpersonlist[i])-1):
        person = sortedpersonlist[i][j]
        personright = sortedpersonlist[i][j+1]
        
        #add size of person picture and spacing to next picture
        #picture size
        vert = famdict[person].vert
        side = famdict[person].side
        g = vert+num_gens_down
        picw = pic_size_w*size_by_degree[g][side]
        #spacing type
        space = spacebetween(person,personright,spacing_couple,spacing_siblings,space_between_families[i])
        
            
        x += picw + space
        #assign xpos to next person
        pic_x_offset[i][j+1] = x
######y offsets
#y offsets based on spacing between generations
y = top_border    
for i in range(len(sortedpersonlist)):    
    ######y offsets
    #start top down
    k = num_gens-1-i
    for j in range(len(sortedpersonlist[k])):
        pic_y_offset[k][j] = y
        
    pich = max_vertical_scaling[k]*pic_size_h
    y += pich + space_between_gens

print('pic_size_w',pic_size_w)
print('pic_size_h',pic_size_h)


#######assign position and pic dimensions to person
#could just set this right away above but I haven't modified the code 
#above yet since implementing the person class. It works this way
for i in range(num_gens):
    for j in range(len(sortedpersonlist[i])):
        person = sortedpersonlist[i][j]

        #picture size
        vert = famdict[person].vert
        side = famdict[person].side
        g = vert+num_gens_down
        picw = pic_size_w*size_by_degree[g][side]
        pich = pic_size_h*size_by_degree[g][side]
        
        famdict[person].picw = picw
        famdict[person].pich = pich
        famdict[person].picx = pic_x_offset[i][j]
        famdict[person].picy = pic_y_offset[i][j]

        
        
######shift kids in lowest generation over so centered under parents if possible
#loop through lower generation
familyassigned = []
kiddict = {}
c=0
for j in range(len(sortedpersonlist[0])):
    p = sortedpersonlist[0][j]
    if p not in familyassigned:
        kiddict[c] = {}
        kiddict[c]['klist'] = [p]
        f=famdict[p].father
        m=famdict[p].mother
        parents = []
        if f is not None:
            parents.append(f)
        if m is not None:
            parents.append(m)
        kiddict[c]['plist'] = parents
        kiddict[c]['pmin'] = min([famdict[par].picx for par in parents])
        kiddict[c]['pmax'] = max([famdict[par].picx 
                                  + famdict[par].picw for par in parents])
        kiddict[c]['kmin'] = famdict[p].picx
        kiddict[c]['kmax'] = famdict[p].picx+famdict[p].picw
        familyassigned.append(p)
        kiddict
        for fam in famdict[p].familylist:
            kiddict[c]['klist'].append(fam)
            if famdict[fam].picx < kiddict[c]['kmin']:
                kiddict[c]['kmin'] = famdict[fam].picx
            if famdict[fam].picx + famdict[fam].picw > kiddict[c]['kmax']:
                kiddict[c]['kmax'] = famdict[fam].picx + famdict[fam].picw 
            familyassigned.append(fam)
        c+=1

for u in range(0,c):
    for i in range(0,c):
        ##shift right
        offset = 0
        kmiddle = kiddict[i]['kmin']+(kiddict[i]['kmax']-kiddict[i]['kmin'])/2
        pmiddle = kiddict[i]['pmin']+(kiddict[i]['pmax']-kiddict[i]['pmin'])/2
        #if additional families right, don't run into them
        if c>i+1:
            while (kiddict[i]['kmax']+offset < (canvas_width-side_border) and 
                   kmiddle + offset < pmiddle and
                   kiddict[i]['kmax']+offset+min_spacing_family < kiddict[i+1]['kmin']):
                offset += 0.001
        else:
            while (kiddict[i]['kmax']+offset < (canvas_width-side_border) and 
                   kmiddle + offset < pmiddle):
                offset += 0.001
        for kid in kiddict[i]['klist']:
            famdict[kid].xOffset(offset)
        kiddict[i]['kmax'] += offset
        kiddict[i]['kmin'] += offset

        ##shift left
        offset = 0
        kmiddle = kiddict[i]['kmin']+(kiddict[i]['kmax']-kiddict[i]['kmin'])/2
        pmiddle = kiddict[i]['pmin']+(kiddict[i]['pmax']-kiddict[i]['pmin'])/2
        #if additional families left, don't run into them
        if i>0:
            while (kiddict[i]['kmin']+offset > side_border and 
                   kmiddle + offset > pmiddle and
                   kiddict[i]['kmin']+offset-min_spacing_family > kiddict[i-1]['kmax']):
                offset -= 0.001
        else:
            while (kiddict[i]['kmin']+offset > side_border and 
                   kmiddle + offset > pmiddle):
                offset -= 0.001
        for kid in kiddict[i]['klist']:
            famdict[kid].xOffset(offset)
        kiddict[i]['kmax'] += offset
        kiddict[i]['kmin'] += offset

    
##########lines between partners
plinelist = []
for i in range(num_gens):
    for j in range(len(sortedpersonlist[i])-1):
        person = sortedpersonlist[i][j]
        personright = sortedpersonlist[i][j+1]
        line = linebetween(person,personright,i,j,sortedpersonlist,r)
        if line is not None:
            plinelist.append(line)
# print('plinelist',plinelist)

linelist = []
#######lines to parents
for i in range(num_gens-1):
    for j in range(len(sortedpersonlist[i])):
        person = sortedpersonlist[i][j]
        line = linetoparents(person,sortedpersonlist,i,j,space_between_gens,r)
        if line is not None:
            linelist.append(line)
# print('linelist',linelist)


######shift lines up some amount if they are going to overlap
#find all yline positions
ylist = []
for line in linelist:
    ylist.append(line[0][1][1])
ylist = np.array(ylist)
ylist = np.unique(ylist)

lc = 0

linelist_ymod = linelist[:]

#loop through all the generation lines
for ygen in ylist:
    c = 0
    y = None
    ##loop through all lines left to right
    for line in linelist:
        x_parent = line[0][2][0]
        x_child = line[0][0][0]
        y_hline = line[0][1][1]
        #if line in the generation being investigated
        if y_hline == ygen:
            #if child to right of mid-canvas
            if x_child > canvas_width*r/2:
                #if first child line being looked at, set comparison to those values
                if y == None:
                    y = y_hline
                    x = x_parent
                    xmax = max(x_child, x_parent)
                else:
                    #if line not running to same line as previous person 
                    if x_parent != x:
                        #if line will overlap with current set comparison line then decrease y to raise line
                        if x_parent <= xmax or x_child <= xmax:
                            c += 1
                            #decrease for all lines that are running to same parent
                            for l,linemod in enumerate(linelist_ymod):
                                x_parentmod = linemod[0][2][0]
                                y_hlinemod = linemod[0][1][1]
                                if x_parentmod == x_parent:
#                                     print('linemod',linemod[0][1])
                                    linelist_ymod[l][0][1] = (linemod[0][1][0], int(linemod[0][1][1]-yspacing*c))
                                    linelist_ymod[l][0][2] = (linemod[0][2][0], int(linemod[0][2][1]-yspacing*c))
#                                     print('linemod',linemod[0][1])
                            #set new comparison values
                            y = y_hline
                            x = x_parent
                            xmax = max(x_child, x_parent)
                        #if line won't overlap then set new comparisons to current person
                        else:
                            y = y_hline
                            x = x_parent
                            xmax = max(x_child, x_parent)
                    #check set new xmax if child is further right than previous max
                    else:
                        if x_child > xmax:
                            xmax = x_child

                            #loop through all the generation lines
linelist_rev = linelist[:]
linelist_rev.reverse()
for ygen in ylist:
    c = 0
    y = None
    ##loop through all lines right to left
    for line in linelist_rev:
        x_parent = line[0][2][0]
        x_child = line[0][0][0]
        y_hline = line[0][1][1]
        #if line in the generation being investigated
        if y_hline == ygen:
            #if child to left of mid-canvas
            if x_child <= canvas_width*r/2:
                #if first child line being looked at, set comparison to those values
                if y == None:
                    y = y_hline
                    x = x_parent
                    xmin = min(x_child, x_parent)
                else:
                    #if line not running to same line as previous person 
                    if x_parent != x:
                        #if line will overlap with current set comparison line then decrease y to raise line
                        if x_parent >= xmin or x_child >= xmin:
                            c += 1
                            #decrease for all lines that are running to same parent
                            for l,linemod in enumerate(linelist_ymod):
                                x_parentmod = linemod[0][2][0]
                                y_hlinemod = linemod[0][1][1]
                                if x_parentmod == x_parent:
                                    linelist_ymod[l][0][1] = (linemod[0][1][0], int(linemod[0][1][1]-yspacing*c))
                                    linelist_ymod[l][0][2] = (linemod[0][2][0], int(linemod[0][2][1]-yspacing*c))
                            #set new comparison values
                            y = y_hline
                            x = x_parent
                            xmin = min(x_child, x_parent)
                        #if line won't overlap then set new comparisons to current person
                        else:
                            y = y_hline
                            x = x_parent
                            xmin = min(x_child, x_parent)
                    #check set new xmin if child is further left than previous min
                    else:
                        if x_child < xmin:
                            xmin = x_child
            #set values across from right to left through list 
            #so that setpoint once child left of canvas center, is previous person
            else:
                y = y_hline
                x = x_parent
                xmin = min(x_child, x_parent)
                      
linelist = linelist_ymod[:]

framefile = filedir + '\\Frames\\frame3.png'
framepic = Image.open(framefile) 
nameplatefile = filedir + '\\Frames\\nameplate.png'
nameplatepic = Image.open(nameplatefile) 

backgroundfile = filedir + '\\Backgrounds\\background10.png'
backgroundpic = Image.open(backgroundfile) 


aplatefile = filedir + '\\Frames\\aplate.png'
aplatepic = Image.open(aplatefile) 

textcolor = (161,117,57)
textcolor = (238,221,164)
#####################################################################
# Image Generation
#####################################################################
###create background
array = np.zeros([canvas_height*r, canvas_width*r, 3], dtype=np.uint8)
# array = np.full([canvas_height*r, canvas_width*r, 3], 255, dtype=np.uint8)
im = Image.fromarray(array)

backgroundpic = backgroundpic.resize((canvas_width*r, canvas_height*r))
im.paste(backgroundpic,(0,0))


#######add lines
d = ImageDraw.Draw(im)
for line in plinelist:
    d.line(line[0],fill=line[1],width=line[2])
for line in linelist:
    d.line(line[0],fill=line[1],width=line[2])
            
###Loop through people and load their photo and resize picture and place where it goes
for i in range(len(sortedpersonlist)): 
    print(i)
    for j in range(len(sortedpersonlist[i])):
        p = sortedpersonlist[i][j]
        ###########load person photo and paste
        directory = filedir + '\\Photos\\Gray\\'
        ext = '.png'
        filename = directory + str(int(p)) + ext
        if not path.exists(filename):
            filename = filedir + '\\Photos\\Gray\\blank-person.png'
        personpic = Image.open(filename) # Can be many different formats.
        #picture resize
        vert = famdict[p].vert
        side = famdict[p].side
        g = vert+num_gens_down
        new_w = int(r*famdict[p].picw)
        new_h = int(r*famdict[p].pich)
        personpic = personpic.resize((new_w, new_h))
        xpos = int(famdict[p].picx*r)
        ypos = int(famdict[p].picy*r)
        
        #blur images for generating shared images
#         personpic = personpic.filter(ImageFilter.GaussianBlur(7))
        
        im.paste(personpic,(xpos,ypos),personpic)
        
        ###########add nameplate
        frame_resized = framepic.resize((new_w, new_h))
        im.paste(frame_resized,(xpos,ypos),frame_resized)
        
        nameplate_resized = nameplatepic.resize((new_w, int(new_w/2)))
        im.paste(nameplate_resized,(xpos,ypos+new_h),nameplate_resized)
        npw, nph = nameplate_resized.size
        
        
        ##########person name text
        pname = famdict[p].name
        ytext = famdict[p].birthdeathtext
        nametext = pname + '\n' + ytext

        #######add text to image
        d = ImageDraw.Draw(im)
        fontsize = getfontsize(fontfile,pname,new_w)
        yfontsize = getfontsize(fontfile,ytext,new_w)
        if yfontsize < fontsize:
            fontsize = yfontsize
        fnt = ImageFont.truetype(fontfile, fontsize)
        
        d.multiline_text((int(xpos+new_w/2),int(ypos+new_h+new_w/4)), nametext, font=fnt, fill=textcolor, 
                          align='center', anchor='mm')
        
        personpic.close()
        
        ##add ancestor text and textframes for upper generation
        if i == len(sortedpersonlist)-1:
            atext = famdict[p].ancestortext
            
            if atext != '':
                ###Place text frame
                a_width = new_w*aframewratio
                a_height = new_h
                incore = famdict[p].incore
                corefm = famdict[p].corefm
                if incore and corefm=='f':
                    text_pos = (int(xpos-new_w*0.05 - a_width/2),int(ypos+(a_height)/2))
                    plate_pos = (int(xpos-new_w*0.05-a_width),int(ypos))
                elif incore and corefm=='m':
                    #check if person to right is another partner, then move text frame over
                    if j+1 < len(sortedpersonlist[i]):
                        if ispartner(p,sortedpersonlist[i][j+1]):
                            text_pos = (int(famdict[sortedpersonlist[i][j+1]].picx*r+new_w*1.05+a_width/2),
                                        int(ypos+(a_height)/2))
                            plate_pos = (int(famdict[sortedpersonlist[i][j+1]].picx*r+new_w + new_w*0.05),int(ypos))
                        else:
                            text_pos = (int(xpos+new_w*1.05+a_width/2),int(ypos+(a_height)/2))
                            plate_pos = (int(xpos+new_w + new_w*0.05),int(ypos))
                    else:
                        text_pos = (int(xpos+new_w*1.05+a_width/2),int(ypos+(a_height)/2))
                        plate_pos = (int(xpos+new_w + new_w*0.05),int(ypos))
                aplate_resized = aplatepic.resize((int(a_width), int(a_height)))
                im.paste(aplate_resized,plate_pos,aplate_resized)

                
                awrap, afontsize = getancfontsize(atext,fontfile,int(a_width),int(a_height))
                wrapper = textwrap.TextWrapper(width=awrap) 
                word_list = wrapper.wrap(text=atext) 
                if len(word_list)>0:
                    caption_new = ''
                    for ii in word_list[:-1]:
                        caption_new = caption_new + ii + '\n'
                    caption_new += word_list[-1]

                    #set font according to longest line of wrapped text
                    fnt = ImageFont.truetype(fontfile, afontsize)
                    d.multiline_text(text_pos, caption_new, font=fnt, 
                                      fill=textcolor, align='center', anchor='mm')

im.save('familytree.png')
im.show()
print('complete')
im.close()

photo_scaling [13.0, 31.0, 20.0, 17.0, 9.0, 16.0]
pic_width 0.9259259259259259 pic_height 1.3888888888888888
space_between_families [3.2453703703703702, 1.1160493827160494, 17.781481481481478, 10.42962962962963, 7.541666666666666, 2.9231481481481483]
space_between_gens 1.1333333333333333
pic_size_w 0.9259259259259259
pic_size_h 1.3888888888888888
0
1
2
3
4
5
complete
